<a href="https://colab.research.google.com/github/pallabi2303/Air-Traffic-Control/blob/main/gpt2_newsSummerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [3]:
from datasets import load_dataset

# Load the 'cnn_dailymail' dataset, version 3.0.0
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:

# Load the splits
train_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")
val_dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation")
test_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")

# Limit the sizes
train_dataset = train_dataset.select(range(1000))
val_dataset = val_dataset.select(range(200))
test_dataset = test_dataset.select(range(200))

In [5]:
def preprocess_function(example):
    # Combine article and summary with a separator
    return {"text": f"Article: {example['article']}\nSummary: {example['highlights']}"}

train_dataset = train_dataset.map(preprocess_function)
val_dataset = val_dataset.map(preprocess_function)
test_dataset = test_dataset.map(preprocess_function)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

train_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id", "text"])
val_tokenized = val_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id", "text"])
test_tokenized = test_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id", "text"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./gpt2-cnn-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

trainer.train()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pallabipati2323 (pallabipati2323-siksha-o-anusandhan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.757500
200,2.678600
300,2.661700
400,2.610900
500,2.646400
600,2.143300
700,2.193100
800,2.168800
900,2.141500
1000,2.109400


In [ ]:
test_results = trainer.evaluate(test_tokenized)
print("Test set evaluation results:", test_results)

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Load the fine-tuned model and tokenizer
generator = pipeline("text-generation", model="gpt2-cnn-finetuned", tokenizer="gpt2-cnn-finetuned", max_length=256)

data = []

for i in tqdm(range(len(test_dataset))):
    article = test_dataset[i]['article']
    reference = test_dataset[i]['highlights']
    prompt = f"Article: {article}\nSummary:"
    generated = generator(prompt, max_new_tokens=60)[0]['generated_text']
    # Extract only the generated summary part
    generated_summary = generated.split("Summary:")[-1].strip()
    data.append({
        "article": article,
        "generated_summary": generated_summary,
        "reference_summary": reference
    })

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("generated_summaries.csv", index=False, encoding="utf-8")

# Download the file
from google.colab import files
files.download("generated_summaries.csv")

In [ ]:
!pip install evaluate
!pip install rouge_score

In [ ]:
import evaluate

# If you followed previous code, you can extract these from your data list or DataFrame
generated_summaries = [row["generated_summary"] for row in data]
reference_summaries = [row["reference_summary"] for row in data]

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print("ROUGE scores:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")